## Improve MNIST with Convolutions

Try to improve MNIST to 99.5% accuracy or more by adding only a single convolutional layer and a single MaxPooling 2D layer to the model 

You should stop training once the accuracy goes above this amount. Training must end once it hits the above metric. If it doesn't, then you'll need to redesign your callback.


In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
# Load the data

# Get current working directory
current_dir = os.getcwd() 

# Append data/mnist.npz to the previous path to get the full path
data_path = os.path.join(current_dir, "data/mnist.npz") 

# Get only training set
(training_images, training_labels), _ = tf.keras.datasets.mnist.load_data(path=data_path) 

## Pre-processing the data

One important step when dealing with image data is to preprocess the data. During the preprocess step you can apply transformations to the dataset that will be fed into your convolutional neural network.

Here we will apply two transformations to the data:
- Reshape the data so that it has an extra dimension. The reason for this 
is that commonly we will use 3-dimensional arrays (without counting the batch dimension) to represent image data. The third dimension represents the color using RGB values. This data might be in black and white format so the third dimension doesn't really add any additional information for the classification process but it is a good practice regardless.


- Normalize the pixel values so that these are values between 0 and 1. We can achieve this by dividing every value in the array by the maximum.

In [7]:
# GRADED FUNCTION: reshape_and_normalize

def reshape_and_normalize(images):
    
    ### START CODE HERE

    # Reshape the images to add an extra dimension
    images = images.reshape(60000, 28, 28, 1)
    
    # Normalize pixel values
    images = images / 255.0
    
    ### END CODE HERE

    return images

Test the function with the next cell:

In [8]:
# Reload the images in case you run this cell multiple times
(training_images, training_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data(path=data_path) 

# Apply your function
training_images = reshape_and_normalize(training_images)

print(f"Maximum pixel value after normalization: {np.max(training_images)}\n")
print(f"Shape of training set after reshaping: {training_images.shape}\n")
print(f"Shape of one image after reshaping: {training_images[0].shape}")


Maximum pixel value after normalization: 1.0

Shape of training set after reshaping: (60000, 28, 28, 1)

Shape of one image after reshaping: (28, 28, 1)


## Defining your callback

Now complete the callback that will ensure that training will stop after an accuracy of 99.5% is reached.

Define callback in such a way that it checks for the metric `accuracy` 

In [16]:
# GRADED CLASS: myCallback
### START CODE HERE

# Remember to inherit from the correct class
class myCallback(tf.keras.callbacks.Callback):
    # Define the method that checks the accuracy at the end of each epoch
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('accuracy')>0.995):
                print("\nReached 99.5% accuracy so cancelling training!")
                self.model.stop_training = True
    

### END CODE HERE


## Convolutional Model

Finally, complete the `convolutional_model` function below. This function should return your convolutional neural network.

**Your model should achieve an accuracy of 99.5% or more before 10 epochs to pass this assignment.**


In [17]:
# GRADED FUNCTION: convolutional_model
def convolutional_model():
    ### START CODE HERE

    # Define the model
    model = tf.keras.models.Sequential([ 
            tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
            tf.keras.layers.MaxPooling2D(2, 2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(10, activation='softmax')
    ]) 

    ### END CODE HERE

    # Compile the model
    model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy']) 
        
    return model

In [21]:
# Save your untrained model
model = convolutional_model()

# Get number of weights
model_params = model.count_params()

# Unit test to limit the size of the model
assert model_params < 1000000, (
    f'Your model has {model_params:,} params. For successful grading, please keep it ' 
    f'under 1,000,000 by reducing the number of units in your Conv2D and/or Dense layers.'
)

# Instantiate the callback class
callbacks = myCallback()

# Train your model (this can take up to 5 minutes)
history = model.fit(training_images, training_labels, epochs=6, callbacks=[callbacks])

Epoch 1/6
1875/1875 [==============================] - 37s 20ms/step - loss: 0.1562 - accuracy: 0.9536
Epoch 2/6
1875/1875 [==============================] - 37s 20ms/step - loss: 0.0515 - accuracy: 0.9846
Epoch 3/6
1875/1875 [==============================] - 37s 20ms/step - loss: 0.0317 - accuracy: 0.9905
Epoch 4/6
1875/1875 [==============================] - 37s 19ms/step - loss: 0.0224 - accuracy: 0.9928
Epoch 5/6
1873/1875 [============================>.] - ETA: 0s - loss: 0.0130 - accuracy: 0.9958
Reached 99.5% accuracy so cancelling training!
1875/1875 [==============================] - 37s 20ms/step - loss: 0.0130 - accuracy: 0.9959


If we see the message that we defined in our callback printed out after less than 10 epochs it means our callback worked as expected. We can also double check by running the following cell:

In [22]:
print(f"Your model was trained for {len(history.epoch)} epochs")

Your model was trained for 5 epochs


In [23]:
if not "accuracy" in history.model.metrics_names:
    print("Use 'accuracy' as metric when compiling your model.")
else:
    print("The metric was correctly defined.")

The metric was correctly defined.
